In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

2025-01-10 08:49:18.481177: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-10 08:49:18.729845: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-10 08:49:21.735342: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-10 08:49:24.413652: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736498967.189051     476 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736498967.62

In [2]:
# Path alla directory principale che contiene le cartelle di LFI
base_dir = "../datasets" 
image_size = (128, 128)  # Risoluzione uniforme per le immagini
batch_size = 8  # Numero di immagini per batch

# Preprocessing con ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1.0/255,          # Normalizza i valori dei pixel tra 0 e 1
    validation_split=0.1      # Dividi il dataset (90% train, 10% validation)
)

# Generatore per il training
train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='input',  # Poiché l'autoencoder utilizza input = output
    subset='training',   # Specifica il sottoinsieme per il training
    color_mode='rgb'     # Specifica se lavorare con immagini RGB
)

# Generatore per la validation
validation_generator = datagen.flow_from_directory(
    base_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='input',  # Poiché l'autoencoder utilizza input = output
    subset='validation', # Specifica il sottoinsieme per la validation
    color_mode='rgb'     # Specifica se lavorare con immagini RGB
)

FileNotFoundError: [Errno 2] No such file or directory: '../dataset'

In [ ]:
# Parametri dell'autoencoder
input_shape = (image_size[0], image_size[1], 3)  # 3 per i canali RGB
input_img = layers.Input(shape=input_shape)

# Encoder
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
encoded = layers.Conv2D(4, (3, 3), activation='relu', padding='same')(x)

# Decoder
x = layers.UpSampling2D((2, 2))(encoded)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x) #Modifica out RGB

# Modello autoencoder
autoencoder = models.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Addestramento
history = autoencoder.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    verbose=2
)

# Visualizzazione della perdita durante l'addestramento
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.title('Loss durante l\'addestramento')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
# Testare l'autoencoder su un batch di immagini
batch = next(train_generator)  # Prendi un batch dal generatore
decoded_images = autoencoder.predict(batch[0])  # Prevedi la ricostruzione

# Visualizzare immagini originali e ricostruite
n = 5  # Numero di immagini da visualizzare
plt.figure(figsize=(10, 4))
for i in range(n):
    # Originale
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(batch[0][i])
    plt.title("Originale")
    plt.axis("off")

    # Ricostruita
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_images[i])
    plt.title("Ricostruita")
    plt.axis("off")
plt.show()

# Salvataggio del modello
autoencoder.save('autoencoder_LFI_model.h5')

In [ ]:
"""import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
# Path alla directory principale che contiene le cartelle di LFI
base_dir = "../dataset" 
image_size = (128, 128)  # Risoluzione uniforme per le immagini

# Funzione per caricare immagini da una directory
def load_images_from_directory(directory, image_size):
    images = []
    for filename in sorted(os.listdir(directory)):
        if filename.endswith('.png'):
            img_path = os.path.join(directory, filename)
            img = load_img(img_path, target_size=image_size, color_mode='rgb')
            img_array = img_to_array(img) / 255.0  # Normalizza tra 0 e 1
            images.append(img_array)
    return np.array(images)

# Caricamento di tutte le immagini da tutte le cartelle
all_images = []
for folder in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder)
    if os.path.isdir(folder_path):
        lfi_images = load_images_from_directory(folder_path, image_size)
        all_images.append(lfi_images)

# Combina tutte le immagini in un unico array
all_images = np.vstack(all_images)
print(f"Dataset di immagini caricato: {all_images.shape}")

# Parametri dell'autoencoder
input_shape = (image_size[0], image_size[1], 3)  # 3 per i canali RGB
input_img = layers.Input(shape=input_shape)

# Encoder
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
encoded = layers.Conv2D(4, (3, 3), activation='relu', padding='same')(x)

# Decoder
x = layers.UpSampling2D((2, 2))(encoded)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# Modello autoencoder
autoencoder = models.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')
# Addestramento
history = autoencoder.fit(all_images, all_images, epochs=50, batch_size=8, validation_split=0.1, verbose=2)

# Visualizzazione della perdita durante l'addestramento
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.title('Loss durante l\'addestramento')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()
# Testare l'autoencoder su una delle immagini
decoded_images = autoencoder.predict(all_images[:10])

# Visualizzare immagini originali e ricostruite
n = 5  # Numero di immagini da visualizzare
plt.figure(figsize=(10, 4))
for i in range(n):
    # Originale
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(all_images[i].astype('float32'))
    plt.title("Originale")
    plt.axis("off")

    # Ricostruita
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_images[i].astype('float32'))
    plt.title("Ricostruita")
    plt.axis("off")
plt.show()

# Salvataggio del modello
autoencoder.save('autoencoder_LFI_model.h5')"""